# Point Inside Circle – Simple Neural Network

In this notebook we build a very small neural network that answers a simple question:

> If we take a point with coordinates (x, y), is this point **inside** a circle of radius 5 with center at (0, 0) or **outside**?

Mathematically a point is inside the circle if:

`x² + y² ≤ 25`

Our goal:

1. Create many example points and mark each one as *inside* or *outside* the circle.
2. Train a neural network to learn this rule from the examples.
3. Check how well the network works on new points.
4. Try a few points by hand.


## 1. Imports

Here we import all tools that we need:

- `numpy` – to work with numbers and arrays;
- `matplotlib` – to draw simple pictures and plots;
- `train_test_split` – to split data into training and test parts;
- `tensorflow` / `keras` – to build and train the neural network.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, models

# So that results are the same each time we run the notebook
np.random.seed(42)
tf.random.set_seed(42)


## 2. Create training data

We will not use any real-world data.  
Instead we will **create our own** simple dataset.

Steps:

1. Take many random points in the square from -6 to 6 on both x and y.
2. For each point `(x, y)` check if it is inside the circle of radius 5.
3. If inside → label is `1.0`.  
   If outside → label is `0.0`.

So we will get two arrays:

- `X` – coordinates of points;
- `y` – answers inside (`1.0`) or outside (`0.0`).

In [ ]:
# How many points we create
N = 20000

# X will have shape (N, 2): [x, y] for each point
X = np.random.uniform(-6, 6, size=(N, 2))


def point_in_circle(x: float, y: float, r: float = 5.0) -> float:
    """Return 1.0 if (x, y) is inside or on a circle with radius r, else 0.0.

    Rule: inside if x**2 + y**2 <= r**2.
    """
    return 1.0 if x**2 + y**2 <= r**2 else 0.0


# Build y: label for each point (1.0 inside, 0.0 outside)
y = np.array([point_in_circle(x, y) for x, y in X], dtype=np.float32)

print("Example point:", X[0])
print("Label (1=inside, 0=outside):", y[0])


### 2.1. Look at the data on a picture (optional)

To better understand the task, let us draw some of the points and the circle.

- Points inside the circle will have one color.
- Points outside will have another color.


In [ ]:
# Take only 1000 random points for drawing
idx = np.random.choice(len(X), size=1000, replace=False)
X_vis = X[idx]
y_vis = y[idx]

inside = X_vis[y_vis == 1.0]
outside = X_vis[y_vis == 0.0]

fig, ax = plt.subplots(figsize=(5, 5))

# Draw outside points
ax.scatter(outside[:, 0], outside[:, 1], s=5, label="outside")

# Draw inside points
ax.scatter(inside[:, 0], inside[:, 1], s=5, label="inside")

# Draw the circle of radius 5
circle = plt.Circle((0, 0), 5, fill=False)
ax.add_artist(circle)

ax.set_aspect('equal', 'box')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Points inside and outside the circle (r = 5)')
ax.legend()
plt.show()


## 3. Split data into training and test parts

We want to know not only how well the network learns the **training** examples,
but also how well it works on **new** points that it has never seen before.

So we split the data into two parts:

- **Training set** – 80% of all points (for learning);
- **Test set** – 20% of all points (for checking).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])


## 4. Build the neural network

We will use a very small network:

- input: 2 numbers (x, y);
- hidden layer 1: 16 neurons with ReLU activation;
- hidden layer 2: 16 neurons with ReLU;
- output layer: 1 neuron with Sigmoid activation.

Short explanation:

- **ReLU** (Rectified Linear Unit) – if the input is positive, it returns this value,
  if negative – it returns 0. This function works well for hidden layers.
- **Sigmoid** – turns any number into a value between 0 and 1. We can think
  of it as a “probability that the point is inside the circle”.

In [ ]:
model = models.Sequential([
    # Input layer: two numbers (x and y)
    layers.Input(shape=(2,)),

    # Hidden layer 1
    layers.Dense(16, activation='relu'),

    # Hidden layer 2
    layers.Dense(16, activation='relu'),

    # Output: one number between 0 and 1 (probability)
    layers.Dense(1, activation='sigmoid')
])

# Tell Keras how we want to train the model
model.compile(
    optimizer='adam',              # how we update weights
    loss='binary_crossentropy',    # loss for 0/1 classification
    metrics=['accuracy']           # we want to see classification accuracy
)

model.summary()


## 5. Train the model

Now we let the neural network **learn** from the training data.

Keras (TensorFlow) does all hard work for us:

1. Sends data through the network (forward pass).
2. Compares predictions with correct answers (loss).
3. Spreads the error backwards (backpropagation).
4. Changes weights a little bit to make the loss smaller.

We repeat this process many times (many **epochs**).

In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,  # 20% of training data used as validation
    epochs=20,
    batch_size=32,
    verbose=1
)


### 5.1. Look at training curves

We can draw how **loss** and **accuracy** change during training.

- Loss should generally go **down**.
- Accuracy should go **up** and become close to 1.0 (100%).

In [ ]:
# Plot loss
plt.figure()
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# Plot accuracy
plt.figure()
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()


## 6. Check the model on the test set

Now we test our network on the points it **has not seen before**.

We will print:

- test loss;
- test accuracy (should be close to 1.0 for this simple task).

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_acc:.4f}")


## 7. Try some points manually

Finally, let us make a small helper function.

It will:

1. Take a point `(x, y)`.
2. Ask the model for probability that the point is inside the circle.
3. Turn this probability into class:

- 1 → inside (or on the border);
- 0 → outside.

We will try a few example points and see what the model says.


In [ ]:
def predict_point(x: float, y: float, threshold: float = 0.5) -> int:
    """Predict whether (x, y) is inside the circle.

    - x, y: coordinates of the point;
    - threshold: if probability >= threshold -> class 1, else class 0.
    """
    point = np.array([[x, y]], dtype=np.float32)  # batch of one point
    prob = model.predict(point, verbose=0)[0, 0]
    predicted_class = 1 if prob >= threshold else 0
    print(f"Point ({x:.2f}, {y:.2f}) -> prob_inside = {prob:.3f}, class = {predicted_class}")
    return predicted_class


# Try a few points
predict_point(1, 1)    # clearly inside
predict_point(5, 0)    # on the circle border
predict_point(6, 0)    # clearly outside
predict_point(4, 4)    # outside
